In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re


In [23]:
driver = webdriver.Chrome()
driver.get("https://www.expedia.com/Hotel-Search?adults=2&d1=2022-01-10&d2=2022-01-11&destination=Mumbai%20%28and%20vicinity%29%2C%20Maharashtra%2C%20India&endDate=2022-01-11&latLong=19.107168%2C72.891596&regionId=6050062&rooms=1&semdtl=&sort=RECOMMENDED&startDate=2022-01-10&theme=&useRewards=false&userIntent=")
for i in range(1):
    time.sleep(5)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")   
    button = driver.find_element_by_xpath('//*[@class="uitk-button uitk-button-medium uitk-button-secondary"]')
    button.click()
soup = str(BeautifulSoup(driver.page_source, 'html.parser'))
soup = BeautifulSoup(soup, 'lxml')

ol = soup.find('ol' , class_  = 'results-list no-bullet')
li = ol.find_all('a', class_='listing__link uitk-card-link')
link_list = []
li.pop(0)
for x,i in enumerate(li):
    link_list.append(i.attrs['href'])
    print(x)
# Save downloaded page as a text file
print(link_list)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
['/Mumbai-Hotels-Niranta-Airport-Transit-Hotel-Lounge-Terminal-2-Arrivals.h11859338.Hotel-Information?chkin=2022-01-10&chkout=2022-01-11&x_pwa=1&rfrr=HSR&pwa_ts=1640615020326&referrerUrl=aHR0cHM6Ly93d3cuZXhwZWRpYS5jb20vSG90ZWwtU2VhcmNo&useRewards=false&rm1=a2&regionId=6050062&destination=Mumbai+%28and+vicinity%29%2C+Maharashtra%2C+India&destType=MARKET&neighborhoodId=553248635974577946&sort=RECOMMENDED&top_dp=75&top_cur=USD&semdtl=&userIntent=&selectedRoomType=201210276&selectedRatePlan=205730812', '/Mumbai-Hotels-Taj-Mahal-Tower.h1351876.Hotel-Information?chkin=2022-01-10&chkout=2022-01-11&x_pwa=1&rfrr=HSR&pwa_ts=1640615020326&referrerUrl=aHR0cHM6Ly93d3cuZXhwZWRpYS5jb20vSG90ZWwtU2VhcmNo&useRewards=false